In [7]:
#check librosa stft and istft
import os
import sys
import numpy as np
import copy

sys.path.insert(0, "/home/anonymous/Desktop/eeg_sampling/modeling")
from dataio import readdata2, readlabels2, writedata
from datafilters import apply_dc_filter, apply_dwt_filter, apply_stfft_filter

In [8]:
#Hyperparams

#DC Filter
enable_dc = True
dc_lowcut = 1.0
dc_highcut = 13.0
dc_order = 2
dc_type = "bandpass"
dc_func_type = "butter"

#DWT Filter
enable_dwt = False
dwt_type = "db2"
dwt_level = 4
dwt_thresh_func = "soft"
dwt_thresh_type = "rigrsure"

#STFT Filter
enable_fft = False
fft_window = 0.50
fft_step = 0.25
fft_thresh = 2.0
fft_set_thresh = 0.0

In [11]:
predataset = readdata2("./../curated/raw-presamples")
dataset = readdata2("./../curated/raw-samples")
labels = readlabels2("./../curated/raw-inputs")
#Constants
fs = 250.0 #Frequency in Hz
sample_time = dataset[0].shape[1]/fs #Total time for sample
presample_time = predataset[0].shape[1]/fs #Total time for sample
num_rows = 360 
num_channels = 8

total_samples = 0
for k in range(0, len(dataset)):
    predataset[k].flags['WRITEABLE'] = True
    dataset[k].flags['WRITEABLE'] = True
    total_samples += dataset[k].shape[0]
    for i in range(0,dataset[k].shape[0]):
        for j in range(0,dataset[k].shape[2]):
            if enable_dc:
                predataset[k][i,:,j] = apply_dc_filter(predataset[k][i,:,j], fs, dc_lowcut, dc_highcut, dc_order, dc_type, dc_func_type)
                dataset[k][i,:,j] = apply_dc_filter(dataset[k][i,:,j], fs, dc_lowcut, dc_highcut, dc_order, dc_type, dc_func_type)
            if enable_dwt:
                predataset[k][i,:,j] = apply_dwt_filter(predataset[k][i,:,j], dwt_type, dwt_level, dwt_thresh_func, dwt_thresh_type)
                dataset[k][i,:,j] = apply_dwt_filter(dataset[k][i,:,j], dwt_type, dwt_level, dwt_thresh_func, dwt_thresh_type)
            if enable_fft:
                predataset[k][i,:,j] = apply_stfft_filter(predataset[k][i,:,j], fs, presample_time, fft_window, fft_step, fft_thresh, fft_set_thresh)
                dataset[k][i,:,j] = apply_stfft_filter(dataset[k][i,:,j], fs, sample_time, fft_window, fft_step, fft_thresh, fft_set_thresh)
            #Normalize
            dataset[k][i,:,j] = dataset[k][i,:,j]/np.linalg.norm(dataset[k][i,:,j])
            #predataset[k][i,:,j] = predataset[k][i,:,j]/np.linalg.norm(predataset[k][i,:,j])
            #np.mean(predataset[k][i,:,j])

In [12]:
test_idx = np.random.random_integers(0,len(dataset)-1,1)
test_dataset = dataset.pop(test_idx[0])
test_labels = labels.pop(test_idx[0])

train_dataset = np.concatenate(dataset)
train_dataset.shape = (total_samples - test_dataset.shape[0], num_rows, num_channels)
train_labels = np.concatenate(labels)

In [16]:
#Write training
writedata("./../curated/train_dataset", train_dataset)
writedata("./../curated/train_labels", train_labels)
#Write validation
#writedata("./curated/valid_dataset", valid_dataset)
#writedata("./curated/valid_labels", valid_labels)
#Write test
writedata("./curated/test_dataset", test_dataset)
writedata("./curated/test_labels", test_labels)
#plt.figure(figsize=(15,8))
#plt.plot(dataset[0,:,0])
#plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './curated/train_dataset'

In [15]:
print(train_labels.shape)
print(train_dataset.shape)
print(test_dataset.shape)

(1250,)
(1250, 360, 8)
(71, 360, 8)
